In [2]:
import json
from scipy.spatial import distance

In [3]:
file_path = '..\\Data\\aligned\\20170207-095506_nb-NO-aihub_aligned.json'

f = open(file_path , encoding='utf-8')
aligned_transcripts = json.load(f)

In [4]:
file_path2 = '..\\Data\\aligned\\20170207_test_alignment.json'

f2 = open(file_path2 , encoding='utf-8')
aligned_transcripts2 = json.load(f2)

In [7]:
edit_distance = 0
for alignment in aligned_transcripts:
    if alignment[0].lower() != alignment[1].lower():
        edit_distance += 1

edit_distance2 = 0
for a in aligned_transcripts2:
    if a[0].lower() != a[1].lower():
        edit_distance2 += 1

print('WER aihub-aligned:', round(edit_distance / len(aligned_transcripts), 3))
print('WER test alignment:', round(edit_distance2 / len(aligned_transcripts2), 3))


WER aihub-aligned: 0.108
WER test alignment: 0.127


In [23]:
# from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("ltgoslo/norbert2")

model = AutoModelForMaskedLM.from_pretrained("ltgoslo/norbert2")

from transformers import pipeline
pipeline = pipeline('feature-extraction', model="ltgoslo/norbert2", tokenizer="ltgoslo/norbert2")
hei = pipeline("far")
hallo = pipeline("pappa")

print(distance.cosine(hei[0][1], hallo[0][1]))


# x = tokenizer("hei", return_tensors='tf')
# y = tokenizer("hallo", return_tensors='tf')
# x = model(**x)
# y = (model(**y))
# cos = distance.cosine(x.logits, y.logits)
# print(cos)

Some weights of the model checkpoint at ltgoslo/norbert2 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at ltgoslo/norbert2 were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.Laye

0


In [30]:
embedded = [[pipeline(x[0].lower()), pipeline(x[1].lower())] for x in aligned_transcripts]

In [45]:
sents = []
new_sent = {}

for alignment, embedding in zip(aligned_transcripts, embedded):
    cos = distance.cosine(embedding[0][0][1], embedding[1][0][1])
    print(alignment[0], alignment[1], cos)
    
    new_sent['cos_sum'] = new_sent.get('cos_sum', 0) + cos

    if alignment[0] not in ['.', '!', '?'] and alignment[0] != ',':
        new_sent['transcript'] = new_sent.get('transcript', '') + ' ' + alignment[0]
        new_sent['gold_standard'] = new_sent.get('gold_standard', '') + ' ' + alignment[1]

    elif alignment[0] == ',':
        new_sent['transcript'] = new_sent.get('transcript', '') + alignment[0]
        new_sent['gold_standard'] = new_sent.get('gold_standard', '') + alignment[1]
        
    else:
        new_sent['transcript'] = new_sent.get('transcript', '') + alignment[0]
        new_sent['gold_standard'] = new_sent.get('gold_standard', '') + alignment[1]
        sents.append(new_sent)
        new_sent = {}

   0
Hei  0.6095063739495219
.  0.6968262069385225
Det  0.6166191679028746
er  0.70615867840054
ikke  0.6188549114606132
det  0.6166191679028746
.  0.6968262069385225
Hei  0.6095063739495219
.  0.6968262069385225
Hei  0.6095063739495219
.  0.6968262069385225
Hei  0.6095063739495219
.  0.6968262069385225
Hei  0.6095063739495219
.  0.6968262069385225
Hei  0.6095063739495219
.  0.6968262069385225
Stortingets Stortingets 0
møte møte 0
er er 0
lovlig lovlig 0
,  0.8184855850860491
satt satt 0
 . 0.6968262069385225
representantene Representantene 0
Fredrik Fredric 0.0816588258193206
helen Helen 0
fredric Fredric 0
holen Holen 0
Bjørdal Bjørdal 0
og og 0
Trond Trond 0
Giske Giske 0
,  0.8184855850860491
som som 0
har har 0
vært vært 0
permitterte permitterte 0
,  0.8184855850860491
har har 0
igjen igjen 0
tatt tatt 0
sete sete 0
 . 0.6968262069385225
følgende Følgende 0
vararepresentanter vararepresentanter 0
tar tar 0
nå nå 0
sete sete 0
nei for Vest Agder . Det er for Vest-Agder 0.647071208

In [46]:
for sent in sents:
    print('Transcript:', sent['transcript'])
    print('Gold standard:', sent['gold_standard'])
    print('SWER:', sent['cos_sum'] / len(sent['transcript'].split(" ")))
    print()

Transcript:    Hei.
Gold standard:   
SWER: 0.3265831452220111

Transcript:  Det er ikke det.
Gold standard:     
SWER: 0.651015626521085

Transcript:  Hei.
Gold standard:  
SWER: 0.6531662904440222

Transcript:  Hei.
Gold standard:  
SWER: 0.6531662904440222

Transcript:  Hei.
Gold standard:  
SWER: 0.6531662904440222

Transcript:  Hei.
Gold standard:  
SWER: 0.6531662904440222

Transcript:  Hei.
Gold standard:  
SWER: 0.6531662904440222

Transcript:  Stortingets møte er lovlig, satt  representantene Fredrik helen fredric holen Bjørdal og Trond Giske, som har vært permitterte, har igjen tatt sete  følgende vararepresentanter tar nå sete nei for Vest Agder fylke, anen Verdal for Akershus fylke, Kjartan berland og are helseth  for Buskerud fylke, lavrans kierulf  for Oslo, ann Kathrine skjørshammer for.
Gold standard:  Stortingets møte er lovlig satt . Representantene Fredric Helen Fredric Holen Bjørdal og Trond Giske som har vært permitterte har igjen tatt sete . Følgende vararepresent